This Notebook evaluates test cases for the 1D Shock-Acoustic-Wave interaction problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
using XDGShock;
Init();


In [2]:
var ass= new MatrixAssembler(1);
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESTSF_ShockAcousticInteraction1D'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D'.


{ Session Count = 15; Grid Count = 15; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

function to plot line outs of the space time solution

In [4]:
(double[] xVals, double[]yVals) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [5]:
var db=OpenOrCreateDatabase(dbPath);
var ses=db.Sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/6/2024 9:31:36 AM	78ffe78d...
#1: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/6/2024 9:31:24 AM	e4f6c055...
#2: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/6/2024 9:31:04 AM	917d7dad...
#3: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:59 PM	39e1f7d3...
#4: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:23 PM	73ae7ed9...
#5: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:47:07 PM	d44f94cb...
#6: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1

In [7]:
var si=db.Sessions.Pick(0);
var ti=si.Timesteps.Last();
//si.Timesteps
si

XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/6/2024 9:31:36 AM	78ffe78d...

# Plotting

define general params for plotting (must substract some eps because, otherwise some points are not located in the space-time grid)

In [8]:
var eps=0.000001;
double xMin=0 +eps;double xMax=3.0-eps; int xRef=100; 
double tMin =0 + eps;int tRef=100;
int nRows=2;

obtain tMax from session name

In [9]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

## Pressure Pertubations

Create .csv files with evaluated fields (for waterfall plots)

In [10]:
using System.Text.RegularExpressions;
foreach(var si in ses){
    string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per.csv";
    Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
    using (StreamWriter writer = new StreamWriter(csvFileName))
            {
            for(int Ti=0;Ti< tRef;Ti++){
                var plot= new Plot2Ddata();
                plot.Xlabel="x";
                plot.Ylabel="p'";
                double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
                //Console.WriteLine(t);
                (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
                plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
                multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
                if (Ti == 0)
                {
                    // Write header row
                    writer.WriteLine($"x/t,{string.Join(",", xVals)}");
                }
                // Write data row
                writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                        
            }
    };
}



doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_a

In [11]:
// Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
//     plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

## Density Pertubations

Create .csv files with evaluated fields (for waterfall plots)

In [12]:
foreach(var si in ses){
        string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="rho_per";
    // Format
    var fmt= new PlotFormat();
    fmt.WithPointSize(0.0);
    fmt.LineColor=LineColors.Red;

    Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
    string csvFileName = si.Name + "_rho_per.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName)){
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="rho'";
            double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
            plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
            multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
            {
                // Write header row
                writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            }
            // Write data row
            writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                    
        }
        
    };                                              
}

doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_a

In [13]:
//  Array.ForEach(multiplotsDensity.Cast<Plot2Ddata>().ToArray(), 
//      plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
//  multiplotsDensity.ToGnuplot().PlotSVG(xRes:1000,yRes:600)